In [1]:
import os 
import sys 

sys.path.insert(0, '..')

In [2]:
from bio_check import Verifier
from pprint import pp

In [3]:
sbml_fp = "../model-examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml"
omex_fp = sbml_fp.replace("/BIOMD0000000012_url.xml", ".omex")
report_fp = omex_fp.replace(".omex", "/reports.h5")
duration = 10
nsteps = 100 
comparison_id = "test_notebook_sbml0"
simulators = ["copasi", "tellurium"]
selection_list = ["cI mRNA"]

In [4]:
verify = Verifier()

{'bio-check-message': 'Hello from the Verification Service API!'}


In [5]:
# inspect public methods and attributes

_dir = dir(verify)

for m in _dir:
    print(m) if not m.startswith("_") else None

data
endpoint_root
export_csv
get_compatible
get_verify_output
select_observables
verify_omex
verify_sbml
visualize


In [6]:
# submit a new verification 

submission_1 = verify.verify_sbml(sbml_filepath=sbml_fp, comparison_id=comparison_id, duration=duration, number_of_steps=nsteps, simulators=simulators)

In [7]:
submission_1

RequestError(error='422 Client Error: Unprocessable Entity for url: https://biochecknet.biosimulations.org/verify-sbml')

In [8]:
submission_2 = verify.verify_omex(omex_filepath=omex_fp, comparison_id=comparison_id, truth=report_fp)

Selection list: None


In [9]:
submission_2

{'job_id': 'utc_comparison_omex_cb1da14e-279c-4f78-b467-4d2d571c2811',
 'status': 'PENDING',
 'timestamp': '2024-07-31 20:33:16.827696',
 'comparison_id': None,
 'path': 'uploads/utc_comparison_omex_cb1da14e-279c-4f78-b467-4d2d571c2811/Elowitz-Nature-2000-Repressilator.omex',
 'simulators': ['amici', 'copasi', 'tellurium'],
 'ground_truth_report_path': None,
 'include_output': True,
 'rTol': None,
 'aTol': None,
 'selection_list': None}

In [10]:
# use job id generated from submission to poll gateway

omex_job_id = submission_2['job_id']

verify.get_verify_output(omex_job_id)

{'content': {'status': 'PENDING',
  'job_id': 'utc_comparison_omex_cb1da14e-279c-4f78-b467-4d2d571c2811',
  'path': 'uploads/utc_comparison_omex_cb1da14e-279c-4f78-b467-4d2d571c2811/Elowitz-Nature-2000-Repressilator.omex',
  'simulators': ['amici', 'copasi', 'tellurium'],
  'timestamp': '2024-07-31 20:33:16.827696',
  'ground_truth_report_path': None,
  'include_outputs': True,
  'rTol': None,
  'aTol': None,
  'selection_list': None}}

In [11]:
# we can call this multiple times until ready

verify.get_verify_output(omex_job_id)

{'content': {'status': 'PENDING',
  'job_id': 'utc_comparison_omex_cb1da14e-279c-4f78-b467-4d2d571c2811',
  'path': 'uploads/utc_comparison_omex_cb1da14e-279c-4f78-b467-4d2d571c2811/Elowitz-Nature-2000-Repressilator.omex',
  'simulators': ['amici', 'copasi', 'tellurium'],
  'timestamp': '2024-07-31 20:33:16.827696',
  'ground_truth_report_path': None,
  'include_outputs': True,
  'rTol': None,
  'aTol': None,
  'selection_list': None}}

In [12]:
output = verify.get_verify_output(omex_job_id)

In [13]:
pp(output)

{'content': {'job_id': 'utc_comparison_01936049-81b2-4ee2-9b05-472cd45a8e46',
             'status': 'IN_PROGRESS',
             'timestamp': '2024-07-31 17:42:19.492427',
             'comparison_id': 'utc_comparison',
             'worker_id': 'ac8a23ef-d810-4a07-9222-2e096adf0ad4'}}


In [10]:
# use selection list to return only one observable

selection_list = ["cI mRNA"]

submission_3 = verify.verify_omex(omex_filepath=omex_fp, comparison_id="notebook_test_omex2", truth=report_fp, selection_list=selection_list)

Selection list: ['cI mRNA']


In [12]:
submission_3

{'job_id': 'utc_comparison_13f90f11-0782-4cfe-9201-12d3354dcf59',
 'status': 'PENDING',
 'timestamp': '2024-07-31 18:19:29.952549',
 'comparison_id': 'utc_comparison',
 'path': 'uploads/utc_comparison_13f90f11-0782-4cfe-9201-12d3354dcf59/Elowitz-Nature-2000-Repressilator.omex',
 'simulators': ['amici', 'copasi', 'tellurium'],
 'ground_truth_report_path': None,
 'include_output': True,
 'rTol': None,
 'aTol': None,
 'selection_list': None}

In [13]:
verify.verify_omex(omex_filepath=omex_fp, comparison_id="notebook_test_omex2", truth=report_fp, selection_list=selection_list)

Selection list: ['cI mRNA']


{'job_id': 'utc_comparison_e50a4894-3ac5-4263-ac2c-734149cfac31',
 'status': 'PENDING',
 'timestamp': '2024-07-31 18:23:05.460839',
 'comparison_id': 'utc_comparison',
 'path': 'uploads/utc_comparison_e50a4894-3ac5-4263-ac2c-734149cfac31/Elowitz-Nature-2000-Repressilator.omex',
 'simulators': ['amici', 'copasi', 'tellurium'],
 'ground_truth_report_path': None,
 'include_output': True,
 'rTol': None,
 'aTol': None,
 'selection_list': None}

In [14]:
import requests 

job_id = 'utc_comparison_e50a4894-3ac5-4263-ac2c-734149cfac31'
piece = f'fetch-results/{job_id}'
endpoint = verify._format_endpoint(piece)
headers = {'Accept': 'application/json'}
try:
    response = requests.get(endpoint, headers=headers)
    verify._check_response(response)
    data = response.json()
    verify.data[job_id] = data
    print(data)
except Exception as e:
    print(str(e))

{'content': {'status': 'PENDING', 'job_id': 'utc_comparison_e50a4894-3ac5-4263-ac2c-734149cfac31', 'path': 'uploads/utc_comparison_e50a4894-3ac5-4263-ac2c-734149cfac31/Elowitz-Nature-2000-Repressilator.omex', 'simulators': ['amici', 'copasi', 'tellurium'], 'comparison_id': 'utc_comparison', 'timestamp': '2024-07-31 18:23:05.460839', 'ground_truth_report_path': None, 'include_outputs': True, 'rTol': None, 'aTol': None, 'selection_list': None}}
